In [1]:
from pathlib import Path
def f2(t):
    print(t)


def print_11():
    cap.show()
    with open(NEWMAN_OUTPUT_FILE, "a") as f:
        f.write(cap.stdout)

In [2]:
def f(i):
    print(i)

In [4]:
%%capture cap --no-stderr
f2(2)
f(1)
NEWMAN_OUTPUT_FILE = Path(".\output_1_NEWMAN.txt")

In [5]:
print_11()

2
1


In [ ]:
cap.show()
with open(NEWMAN_OUTPUT_FILE, "a") as f:
    f.write(cap.stdout)

In [ ]:
# with open(NEWMAN_OUTPUT_FILE, "a") as f:
#     f.write(cap.stdout)
# # NEWMAN_OUTPUT.write(cap.stdout)


# Question 1